# 1D Heat Equation: Numerical Solution of Thermal Diffusion

## Theoretical Foundation

The **heat equation** is a fundamental partial differential equation (PDE) that describes the distribution of heat (or temperature variations) in a given region over time. It is a paradigmatic example of a **parabolic PDE** and serves as the mathematical foundation for understanding diffusion processes in physics, chemistry, and biology.

### The One-Dimensional Heat Equation

In one spatial dimension, the heat equation takes the form:

$$\frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}$$

where:
- $u(x, t)$ is the temperature at position $x$ and time $t$
- $\alpha$ is the **thermal diffusivity** of the material (units: $\text{m}^2/\text{s}$)

The thermal diffusivity is defined as:

$$\alpha = \frac{k}{\rho c_p}$$

where $k$ is thermal conductivity, $\rho$ is density, and $c_p$ is specific heat capacity.

### Physical Interpretation

The heat equation embodies **Fourier's law of heat conduction**: heat flows from regions of high temperature to regions of low temperature at a rate proportional to the temperature gradient. The second spatial derivative $\partial^2 u / \partial x^2$ measures the local curvature of the temperature profile—regions where the temperature is locally higher than their surroundings will cool down, while local minima will warm up.

### Boundary and Initial Conditions

To solve the heat equation, we require:

1. **Initial condition**: $u(x, 0) = f(x)$ — the temperature distribution at $t = 0$
2. **Boundary conditions**: We use **Dirichlet conditions** (fixed temperature at boundaries):
   - $u(0, t) = T_L$ (left boundary)
   - $u(L, t) = T_R$ (right boundary)

### Numerical Method: Finite Difference (FTCS Scheme)

We employ the **Forward-Time Central-Space (FTCS)** explicit finite difference method. Discretizing space and time with step sizes $\Delta x$ and $\Delta t$:

$$u_i^{n+1} = u_i^n + r\left(u_{i+1}^n - 2u_i^n + u_{i-1}^n\right)$$

where $r = \alpha \Delta t / (\Delta x)^2$ is the **Fourier number** (or mesh ratio).

### Stability Criterion

The FTCS scheme is **conditionally stable**. The **von Neumann stability analysis** yields the requirement:

$$r = \frac{\alpha \Delta t}{(\Delta x)^2} \leq \frac{1}{2}$$

Violating this condition leads to numerical instability with exponentially growing oscillations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

## Problem Setup

We consider a rod of length $L = 1$ m with the following conditions:

- **Initial condition**: A Gaussian temperature pulse centered at $x = 0.5$
  $$u(x, 0) = \exp\left(-\frac{(x - 0.5)^2}{0.01}\right)$$
  
- **Boundary conditions**: $u(0, t) = u(1, t) = 0$ (fixed at zero temperature)

- **Thermal diffusivity**: $\alpha = 0.01 \, \text{m}^2/\text{s}$

In [ ]:
# Physical and numerical parameters
L = 1.0              # Length of the rod (m)
alpha = 0.01         # Thermal diffusivity (m^2/s)
T_total = 2.0        # Total simulation time (s)

# Spatial discretization
Nx = 101             # Number of spatial points
dx = L / (Nx - 1)    # Spatial step size
x = np.linspace(0, L, Nx)

# Temporal discretization (ensuring stability: r <= 0.5)
r = 0.4              # Fourier number (stability parameter)
dt = r * dx**2 / alpha
Nt = int(T_total / dt) + 1
t = np.linspace(0, T_total, Nt)

print(f"Spatial step size: dx = {dx:.4f} m")
print(f"Time step size: dt = {dt:.6f} s")
print(f"Number of time steps: Nt = {Nt}")
print(f"Fourier number: r = {r:.2f} (stable if r <= 0.5)")

In [ ]:
# Initial condition: Gaussian temperature pulse
def initial_condition(x):
    """Gaussian pulse centered at x = 0.5"""
    return np.exp(-(x - 0.5)**2 / 0.01)

# Initialize temperature array
u = np.zeros((Nt, Nx))
u[0, :] = initial_condition(x)

# Apply boundary conditions (Dirichlet: u = 0 at boundaries)
u[:, 0] = 0.0
u[:, -1] = 0.0

## FTCS Algorithm Implementation

The explicit finite difference scheme updates each interior point using the values from the previous time step. This is computationally efficient but requires careful attention to the stability criterion.

In [ ]:
# Time-stepping loop using FTCS scheme
for n in range(0, Nt - 1):
    for i in range(1, Nx - 1):
        u[n+1, i] = u[n, i] + r * (u[n, i+1] - 2*u[n, i] + u[n, i-1])
    
    # Boundary conditions are already set (remain zero)

print("Simulation complete!")

## Visualization of Results

We visualize the temperature evolution in three complementary ways:
1. **Temporal snapshots**: Temperature profiles at selected times
2. **Space-time heatmap**: Full $u(x, t)$ field
3. **3D surface**: Temperature as a function of both space and time

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(14, 10))

# Plot 1: Temperature profiles at different times
ax1 = fig.add_subplot(2, 2, 1)
time_indices = [0, Nt//8, Nt//4, Nt//2, Nt-1]
colors = plt.cm.viridis(np.linspace(0, 1, len(time_indices)))

for idx, ti in enumerate(time_indices):
    ax1.plot(x, u[ti, :], color=colors[idx], linewidth=2, 
             label=f't = {t[ti]:.2f} s')

ax1.set_xlabel('Position x (m)', fontsize=12)
ax1.set_ylabel('Temperature u(x,t)', fontsize=12)
ax1.set_title('Temperature Profiles at Different Times', fontsize=14)
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
ax1.set_xlim([0, L])

# Plot 2: Space-time heatmap
ax2 = fig.add_subplot(2, 2, 2)
X, T = np.meshgrid(x, t)
contour = ax2.contourf(X, T, u, levels=50, cmap='hot')
cbar = plt.colorbar(contour, ax=ax2)
cbar.set_label('Temperature', fontsize=10)
ax2.set_xlabel('Position x (m)', fontsize=12)
ax2.set_ylabel('Time t (s)', fontsize=12)
ax2.set_title('Temperature Evolution (Space-Time)', fontsize=14)

# Plot 3: 3D surface plot
ax3 = fig.add_subplot(2, 2, 3, projection='3d')
# Subsample for clarity
stride_t = max(1, Nt // 50)
stride_x = max(1, Nx // 50)
X_sub, T_sub = np.meshgrid(x[::stride_x], t[::stride_t])
u_sub = u[::stride_t, ::stride_x]

surf = ax3.plot_surface(X_sub, T_sub, u_sub, cmap='coolwarm', 
                        edgecolor='none', alpha=0.9)
ax3.set_xlabel('Position x (m)', fontsize=10)
ax3.set_ylabel('Time t (s)', fontsize=10)
ax3.set_zlabel('Temperature', fontsize=10)
ax3.set_title('3D Temperature Surface', fontsize=14)
ax3.view_init(elev=25, azim=45)

# Plot 4: Energy dissipation (total heat content over time)
ax4 = fig.add_subplot(2, 2, 4)
total_heat = np.trapz(u, x, axis=1)
ax4.plot(t, total_heat, 'b-', linewidth=2)
ax4.set_xlabel('Time t (s)', fontsize=12)
ax4.set_ylabel('Total Heat Content', fontsize=12)
ax4.set_title('Heat Dissipation Through Boundaries', fontsize=14)
ax4.grid(True, alpha=0.3)

# Add annotation about energy loss
initial_heat = total_heat[0]
final_heat = total_heat[-1]
heat_loss = (1 - final_heat/initial_heat) * 100
ax4.annotate(f'Heat lost: {heat_loss:.1f}%', 
             xy=(t[-1]*0.6, total_heat[-1]*1.5),
             fontsize=10, color='red')

plt.tight_layout()

# Save the figure
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
print("Figure saved as 'plot.png'")

plt.show()

## Analysis and Physical Interpretation

### Key Observations

1. **Diffusive Spreading**: The initial Gaussian pulse spreads out over time, demonstrating the fundamental nature of diffusion—sharp features are smoothed.

2. **Peak Reduction**: The maximum temperature decreases monotonically as heat diffuses outward and exits through the boundaries.

3. **Energy Conservation**: With Dirichlet boundary conditions (fixed at zero), heat continuously flows out of the domain. The total thermal energy $\int_0^L u(x,t) \, dx$ decreases over time.

4. **Asymptotic Behavior**: As $t \to \infty$, the solution approaches the steady state $u(x) = 0$, consistent with the boundary conditions.

### Characteristic Diffusion Time

The **diffusion time scale** for the system is:

$$\tau_D = \frac{L^2}{\alpha}$$

For our parameters: $\tau_D = \frac{1^2}{0.01} = 100$ s

This explains why significant evolution occurs over our 2-second simulation window—the characteristic time is much longer, so we observe only the initial transient behavior.

In [ ]:
# Calculate and display characteristic scales
tau_D = L**2 / alpha
print(f"Characteristic diffusion time: τ_D = {tau_D:.1f} s")
print(f"Simulation time / τ_D = {T_total/tau_D:.2%}")
print(f"\nAt t = {T_total} s:")
print(f"  Peak temperature: {np.max(u[-1, :]):.4f}")
print(f"  Peak location: x = {x[np.argmax(u[-1, :])]:.2f} m")

## Conclusion

This notebook demonstrated the numerical solution of the 1D heat equation using the explicit FTCS finite difference method. Key concepts covered include:

- The physical basis of thermal diffusion (Fourier's law)
- Discretization of the heat equation using finite differences
- The critical importance of the stability criterion $r \leq 0.5$
- Visualization techniques for understanding diffusion dynamics

### Extensions

Natural extensions of this work include:
- **Implicit methods** (Crank-Nicolson) for unconditional stability
- **Neumann boundary conditions** (insulated boundaries)
- **Variable thermal diffusivity** $\alpha(x)$
- **Source terms** (heat generation/absorption)
- **Higher dimensions** (2D and 3D heat equation)